# DynamoDBのテーブルから項目数を取得する

以下コード内のAWS接続設定は、ローカルPC上で `localstack` に接続する設定にしています。


In [1]:
!python -V

Python 3.8.3


In [2]:
!pip -V

pip 20.2.2 from /usr/local/lib/python3.8/site-packages/pip (python 3.8)


In [3]:
!pip install boto3

In [4]:
import boto3

## AWS config設定

In [5]:
aws_config = {
   "aws_access_key_id": "dummy",
   "aws_secret_access_key": "dummy",
   "region_name": "ap-northeast-1",
   "endpoint_url": "http://localstack:4566"
}

## client インスタンスを生成

In [6]:
client = boto3.client(service_name="dynamodb", **aws_config)
dynamodb = boto3.resource(service_name="dynamodb", **aws_config)

## テーブルリスト取得

In [7]:
table_list = client.list_tables()
table_list['TableNames']

['LocalTest']

## テーブルの件数取得

In [8]:
# `LocalTest` テーブルの件数取得
table = dynamodb.Table("LocalTest")

### DynamoDB.Table.item_count

In [9]:
# `item_count` は6時間ごとに更新された値が取得されます（ローカルの場合リアルタイムで取得できます）
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table.item_count
table.item_count

100000

### DynamoDB.Table.scan

In [10]:
# `scan` で一度に取得できるデータサイズは最大1MBに制限されています。
# 条件なしで `COUNT` を取得しても1MBまでの件数しか取得できません。
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#DynamoDB.Table.scan
ret = table.scan(Select='COUNT')
ret['Count']

16298

### DynamoDB.Table.scan with Pagenator

In [16]:
# `item_count` および `scan` の制限を回避して全項目を取得する方法として、 `scan` の説明にあるようにページ分割をして取得する方法があります。
# 以下、全項目数（レコード数）を取得する例を示していますが、 `page["Items"]` で項目も取得できます。
total = 0
total_steps = []
paginator = client.get_paginator("scan")
for page in paginator.paginate(TableName="LocalTest", PaginationConfig={
        'MaxItems': 500000 # 全項目数より大きな値を設定
    }):
    total += page['Count']
    total_steps.append(str(total))
    print('.', end='')
print('\n' + '->'.join(total_steps))

.......
16298->32589->48879->65183->81475->97776->100000
